In [2]:
import os

In [3]:
%pwd

'd:\\Courses\\Krish_Naik\\Chicken_Disease_Classification\\research'

In [4]:
# move one folder up
os.chdir("../")

In [5]:
%pwd

'd:\\Courses\\Krish_Naik\\Chicken_Disease_Classification'

Workflow - (4) Update Entity

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [7]:
# this class is a data class and not python class
# the parameters are the return type of the class
# this is similar to data_ingestion present in config.yaml
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from chicken_disease_classification.constants import *
from chicken_disease_classification.utils.common import read_yaml, create_directories

Create Artifacts root folder and then create a sub folder called data_ingestion inside it

In [17]:
class ConfigurationManager:
    def __init__(
        self, 
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH
    ):
        self.config_file_path = read_yaml(config_file_path)
        self.params_file_path = read_yaml(params_file_path)
        
        # create directory -> "articfacts"; artifacts_root is present in config.yaml
        create_directories([self.config_file_path.artifacts_root])
    
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        # data_ingestion variable is present in config.yaml
        config = self.config_file_path.data_ingestion
        
        # create sub folder "data_ingestion" in "artifacts" folder
        create_directories([config.root_dir])
        
        # return DataIngestionConfig object
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

Update the Components

In [18]:
import os
# to download the file
import urllib.request as request
# to unzip the file
import zipfile
from chicken_disease_classification import logger
from chicken_disease_classification.utils.common import get_size


Download the file from URL and extract the zip insde the sub folder data_ingestion which we created above

In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    # download the file from the source url
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            file_name, headers = request.urlretrieve(url = self.config.source_url, filename = self.config.local_data_file)
            logger.info(f"{file_name} downloaded successfully with the following headers: {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    
    # unzip the file after downloading
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory.
        
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

Create Pipeline that will download the zip file from github URL and extract the zip

In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-06-06 10:21:46,942: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-06 10:21:46,947: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-06 10:21:46,951: INFO: common: Created directory at: artifacts]
[2023-06-06 10:21:46,954: INFO: common: Created directory at: artifacts/data_ingestion]
[2023-06-06 10:21:49,302: INFO: 11847839: artifacts/data_ingestion/data.zip downloaded successfully with the following headers: Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9688:3F6B:15B00CA:19AC884:647F6662
Accept-Ranges: bytes
Date: Tue, 06 Jun 2023 17:21:47 GMT
Via: 1.1 varnish
X-Served-By: cache-pao-